In [15]:
import mlflow
from mlflow.tracking import MlflowClient

# Set MLflow tracking URI
mlflow.set_tracking_uri("file:///Users/chloe/PycharmProjects/bnb/bnb_repo/mlruns")
client = MlflowClient()

experiment_names = [
    "airbnb_pricing_models",
    "airbnb_pricing_models_1",
    "airbnb_pricing_models_2",
    "airbnb_pricing_models_3"
]

qualified_runs = []

for exp_name in experiment_names:
    experiment = client.get_experiment_by_name(exp_name)
    if experiment is None:
        print(f"Experiment '{exp_name}' not found.")
        continue

    experiment_id = experiment.experiment_id
    runs = client.search_runs(
        experiment_ids=[experiment_id],
        order_by=["metrics.rmse_dollar ASC"],
        max_results=10
    )

    for run in runs:
        metrics = run.data.metrics
        r2 = metrics.get("r2_score", 0)
        rmse_dollar = metrics.get("rmse_dollar", float("inf"))
        rmse_raw = metrics.get("rmse_raw", float("inf"))
        mae = metrics.get("mae_log", float("inf"))

        if r2 > 0.6:
            score = 0.5 * rmse_dollar + 0.3 * rmse_raw + 0.2 * mae
            qualified_runs.append((run, exp_name, score))

# Select best run by composite score
if qualified_runs:
    best_run, best_experiment, best_score = sorted(qualified_runs, key=lambda x: x[2])[0]

    print(" Registering best model from experiment:", best_experiment)
    print("Run ID:", best_run.info.run_id)
    print(f"R²: {best_run.data.metrics['r2_score']:.4f}")
    print(f"Composite Score: {best_score:.4f}")
    print(f"RMSE Real ($): {best_run.data.metrics['rmse_dollar']:.2f}")

    model_uri = f"runs:/{best_run.info.run_id}/model"
    mlflow.register_model(model_uri, "airbnb_best_model_in_range")
    print("Model registered as 'airbnb_best_model_in_range'")

else:
    print("No suitable runs found with R² > 0.6.")


Registered model 'airbnb_best_model_in_range' already exists. Creating a new version of this model...
2025/06/17 12:16:22 WARNING mlflow.tracking._model_registry.fluent: Run with id a0f04a24ffeb46c4b20a6045f988de9d has no artifacts at artifact path 'model', registering model based on models:/m-eadd16085d1e47c0b98fa14715960269 instead


 Registering best model from experiment: airbnb_pricing_models_2
Run ID: a0f04a24ffeb46c4b20a6045f988de9d
R²: 0.6405
Composite Score: 35.6629
RMSE Real ($): 71.00
Model registered as 'airbnb_best_model_in_range'


Created version '3' of model 'airbnb_best_model_in_range'.
